# Linebackers
- Jan 19, 2018

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from MUTScraper import Player, PlayerHandler
%load_ext autoreload
%autoreload 2

In [2]:
import plotly as py
import plotly.plotly as py
import plotly.graph_objs as go
import pandas as pd
from ipywidgets import interactive, HBox, VBox, widgets, interact

In [3]:
ph = PlayerHandler(position='LB', min_ovr=90, date='jan19', pages=4)
ph.handle_players()

 100 player links gathered.


In [5]:
lb_groups = ph.player_df.groupby('Position')

df_dict = {}

for name, group in lb_groups:
    df_dict.update({name: group})

In [6]:
pdf = ph.player_df
pdf.head()

,ACC,AGI,AWR,BKS,CIT,CTH,FNM,JMP,MCV,POW,...,DL Swim Move,High Motor,LB Style,Penalty,Plays Ball in Air,Strips Ball,HtWt,OVR,Position,Team
Bobby Wagner,93,89,98,93,58,81,72,95,80,96,...,No,Yes,Cover LB,Disciplined,Aggressive,No,"Ht: 6' 0"" Wt: 245",98,MLB,Seattle Seahawks
Willie McGinest,88,80,97,92,61,67,92,78,75,91,...,No,Yes,Balanced,Norma,Balanced,Yes,"Ht: 6' 5"" Wt: 270",97,ROLB,New England Patriots
Derrick Brooks,91,88,95,84,69,82,83,81,82,87,...,No,Yes,Balanced,Norma,Balanced,No,"Ht: 6' 0"" Wt: 235",96,ROLB,Tampa Bay Buccaneers
Justin Houston,89,86,95,93,28,52,93,78,68,92,...,Yes,Yes,Pass Rush,Disciplined,Conservative,Yes,"Ht: 6' 3"" Wt: 258",96,LOLB,Kansas City Chiefs
Darius Leonard,91,88,90,89,60,74,87,94,78,90,...,Yes,Yes,Balanced,Norma,Balanced,Yes,"Ht: 6' 2"" Wt: 234",96,ROLB,Indianapolis Colts


In [14]:
# height
htwt = pdf['HtWt'].str.split('Wt: ')
ht = htwt.map(lambda x: x[0])
wt = htwt.map(lambda x: x[1])

ht2 = ht.str.lstrip('Ht: ').str.rstrip('" ').str.split("' ")
pdf['Ht'] = ht2.map(lambda x: int(x[0]) * 12 + int(x[1].strip()))
pdf['Wt'] = wt.astype(int)
pdf['OVR'] = pdf['OVR'].astype(int)

lb_groups = pdf.groupby('Position')

df_dict = {}

for name, group in lb_groups:
    df_dict.update({name: group})

In [44]:
pdf.to_csv('mut19_linebackers_jan19.csv')

In [30]:
def attribute_restriction(dat, attrib, rating):
    df = dat.copy()
    df = df[ df[attrib] >= rating]
    return df

In [40]:
pdf2 = attribute_restriction(pdf, 'AGI', 87)
pdf2 = attribute_restriction(pdf2, 'SPD', 85)

df = pdf2

color_map = {'ROLB': 'rgb(230,0,230)', 'MLB': 'rgb(230,230,230)', 'LOLB': 'rgb(240,5,30)'}

attrib_list = ['SPD', 'Ht', 'AGI', 'JMP']

def get_trace(pos):
    df0 = df_dict.get(pos)

    trace = go.Splom(dimensions=[dict(label=attrib, values=df0[attrib]) for attrib in attrib_list],
                text=df0.index.values,
                name=pos,
                marker=dict(size=6,
                            showscale=False,
                            line=dict(width=0.5,
                                      color=color_map.get(pos)))
                )
    return trace

In [45]:
# Trace
trace0 = get_trace('ROLB')
trace1 = get_trace('MLB')
trace2 = get_trace('LOLB')
traces = [trace0, trace1, trace2]

# Axis
axis = dict(showline=True,
          zeroline=False,
          gridcolor='#fff',
          ticklen=4)

layout = go.Layout(
    title='MUT19 Offensive Lineman (90+ OVR)',
    dragmode='select',
    width=1000,
    height=800,
    autosize=False,
    hovermode='closest',
    plot_bgcolor='rgba(240,240,240, 0.95)',
    xaxis1=dict(axis), xaxis2=dict(axis), xaxis3=dict(axis), xaxis4=dict(axis),
    xaxis5=dict(axis), #xaxis6=dict(axis), xaxis7=dict(axis), xaxis8=dict(axis),
    yaxis1=dict(axis), yaxis2=dict(axis), yaxis3=dict(axis), yaxis4=dict(axis),
    yaxis5=dict(axis), #yaxis6=dict(axis), yaxis7=dict(axis), yaxis8=dict(axis)
)

for trace in traces:
    trace['diagonal'].update(visible=False)
    trace['showupperhalf']=False

fig = dict(data=traces, layout=layout)

py.iplot(fig)

In [39]:

f = go.FigureWidget(data=fig['data'], layout=fig['layout'])

slider = widgets.IntSlider(
    value=90,
    min=89,
    max=99,
    step=1,
    description='SPD',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)


slider.layout.width = '600px'

def update_range():
    f.data[0].z = np.cos(x * yt * frequency / 10.0) + np.sin(x * yt * frequency / 10.0) * 2


# display the FigureWidget and slider with center justification
vb = VBox((f, interactive(update_range, y=slider)))
vb.layout.align_items = 'center'
vb
'''
py.iplot(fig)